### Домашнее задание №5.
1. Для нашего пайплайна (Case1) поэкспериментировать с разными моделями:
    - бустинг;
    - логистическая регрессия.
2. Отобрать лучшую модель по метрикам (какая по вашему мнению здесь наиболее подходящая ML-метрика).
3. Для отобранной модели (на отложенной выборке) сделать оценку экономической эффективности при тех же вводных, как в вопросе 2:
    - 1 доллар на удержание;
    - 2 доллара - с каждого правильно классифицированного (True Positive).
4. *Провести подбор гиперпараметров лучшей модели по итогам 2-3.
5. *Еще раз провести оценку экономической эффективности.

In [1]:
import pandas as pd
import numpy as np

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import precision_recall_curve, roc_auc_score, confusion_matrix

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
class FeatureSelector(BaseEstimator, TransformerMixin):
    
    def __init__(self, column):
        self.column = column

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.column]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        return pd.get_dummies(X, prefix=self.key)[self.columns]

- Разобьем признаки на группы: вещественные, категориальные и бинарные.

In [3]:
cat_cols = ['Geography', 'Gender', 'Tenure']
cont_cols = ['CreditScore', 'Age', 'Balance', 'NumOfProducts', 'EstimatedSalary']
bin_cols = ['HasCrCard', 'IsActiveMember']

- Напишем пайплайны для всех признаков.

In [4]:
final_transformers = []

for cat_col in cat_cols:
    cat_transformer = Pipeline([
                ('selector', FeatureSelector(column=cat_col)),
                ('ohe', OHEEncoder(key=cat_col))
            ])
    
    final_transformers.append((cat_col, cat_transformer))
    
for cont_col in cont_cols:
    cont_transformer = Pipeline([
                ('selector', NumberSelector(key=cont_col)),
                ('scaler', StandardScaler())
            ])
    
    final_transformers.append((cont_col, cont_transformer))

for bin_col in bin_cols:
    bin_transformer = Pipeline([
                ('selector', NumberSelector(key=bin_col))
            ])
    
    final_transformers.append((bin_col, bin_transformer))

In [5]:
feats = FeatureUnion(final_transformers)

feature_processing = Pipeline([('feats', feats)])

- Загрузим датасет и разобьем его на train и test.

In [6]:
df = pd.read_csv("churn_data.csv")
df.head(3)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1


In [7]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(['Exited'], axis=1), df['Exited'], random_state=23)

- Построим модели логистической регресси, случайного леса и градиентного бустинга. Для двух последних предварительно с помощью GridSearchCV подберем лучшие гиперпараметры.

__Логистическая регрессия__

In [8]:
lr = Pipeline([('features', feats),
               ('classifier', LogisticRegression(random_state=23))])

cv_scores = cross_val_score(lr, X_train, y_train, cv=5, scoring='roc_auc')
cv_score = np.mean(cv_scores)
cv_score_std = np.std(cv_scores)
print(f'CV score is {cv_score:.5f} +- {cv_score_std:.5f}')

CV score is 0.76337 +- 0.00972


__Случайный лес__

In [9]:
rfc_def = Pipeline([('features', feats),
                    ('classifier', RandomForestClassifier(random_state=23))])

In [10]:
params_rfc = {
    'classifier__max_features': [0.2, 0.3, 0.5, 0.7],
    'classifier__min_samples_leaf': [1, 2, 3, 4, 5],
    'classifier__class_weight':[{0: 1, 1: 4}, 'balanced_subsample']
}

In [11]:
%%time
rfc_grid = GridSearchCV(rfc_def,
                        param_grid=params_rfc,
                        cv=5,
                        scoring='f1')

rfc_search = rfc_grid.fit(X_train, y_train)

Wall time: 4min 53s


In [12]:
rfc_search.best_params_

{'classifier__class_weight': 'balanced_subsample',
 'classifier__max_features': 0.3,
 'classifier__min_samples_leaf': 4}

In [13]:
rfc_search.best_score_

0.6274307437272811

In [14]:
rfc = Pipeline([('features', feats),
                ('classifier', RandomForestClassifier(min_samples_leaf=4,
                                                      max_features=0.3,
                                                      class_weight='balanced_subsample',
                                                      random_state=23))])

cv_scores = cross_val_score(rfc, X_train, y_train, cv=5, scoring='roc_auc')
cv_score = np.mean(cv_scores)
cv_score_std = np.std(cv_scores)
print(f'CV score is {cv_score:.5f} +- {cv_score_std:.5f}')

CV score is 0.85896 +- 0.01174


__Градиентный бустинг__

In [15]:
gbc_def = Pipeline([('features', feats),
                    ('classifier', GradientBoostingClassifier(random_state=23))])

In [16]:
params_gbc = {
    'classifier__learning_rate': [0.05, 0.08, 0.1],
    'classifier__n_estimators': [100, 120, 140, 160, 180],
    'classifier__max_depth': [2, 3, 4, 5, 6]
}

In [17]:
%%time
gbc_grid = GridSearchCV(gbc_def,
                        param_grid=params_gbc,
                        cv=5,
                        scoring='f1')

gbc_search = gbc_grid.fit(X_train, y_train)

Wall time: 14min 43s


In [18]:
gbc_search.best_params_

{'classifier__learning_rate': 0.08,
 'classifier__max_depth': 5,
 'classifier__n_estimators': 120}

In [19]:
gbc_search.best_score_

0.6003868748109693

In [20]:
gbc = Pipeline([('features', feats),
                ('classifier', GradientBoostingClassifier(learning_rate=0.08,
                                                          n_estimators=120,
                                                          max_depth=5,
                                                          random_state=23))])

cv_scores = cross_val_score(gbc, X_train, y_train, cv=5, scoring='roc_auc')
cv_score = np.mean(cv_scores)
cv_score_std = np.std(cv_scores)
print(f'CV score is {cv_score:.5f} +- {cv_score_std:.5f}')

CV score is 0.85946 +- 0.01073


- Обучим каждую модель на отложенной выборке и посчитаем ML-метрики, собрав в общую таблицу.

In [21]:
pivot_metric_table = []
models_pred_proba_values = []

for model in ['lr', 'rfc', 'gbc']:
    eval(f'{model}.fit(X_train, y_train)')
    preds = eval(f'{model}.predict_proba(X_test)[:, 1]')
    models_pred_proba_values.append(preds)
    
    precision, recall, thresholds = precision_recall_curve(y_test, preds)
    precision[(precision == 0) & (recall == 0)] = np.e-10
    fscore = (2 * precision * recall) / (precision + recall)
    
    ix = np.argmax(fscore)
    
    pivot_metric_table.append((fscore[ix], precision[ix], recall[ix], thresholds[ix], roc_auc_score(y_test, preds)))

pivot_metric_table = pd.DataFrame(pivot_metric_table)
pivot_metric_table.columns = ['F score', 'Precision', 'Recall', 'Optimal threshold', 'ROC AUC score']
pivot_metric_table['Classifier'] = ['Logistic Regression', 'Random Forest', 'Gradient Boosting']
pivot_metric_table.set_index('Classifier', inplace=True)

In [22]:
pivot_metric_table

,F score,Precision,Recall,Optimal threshold,ROC AUC score
Classifier,,,,,
Logistic Regression,0.481640,0.421525,0.561753,0.265910,0.766747
Random Forest,0.626709,0.663697,0.593625,0.518071,0.863991
Gradient Boosting,0.624465,0.547226,0.727092,0.220511,0.870244


- Произведем оценку экономической эффективности моделей с порогом срабатывания, соответствующим лучшим показателям ML-метрик.

In [23]:
def get_business_profit(y_test, y_preds, treshold=0.5,
                       retain_cost=1, spec_profit=2):
    
    cnf_matrix = confusion_matrix(y_test, y_preds > treshold)
    TN = cnf_matrix[0][0]
    FN = cnf_matrix[1][0]
    TP = cnf_matrix[1][1]
    FP = cnf_matrix[0][1]
    
    return spec_profit * TP - retain_cost * (TP + FP)

In [24]:
profit_lr = get_business_profit(y_test, models_pred_proba_values[0], pivot_metric_table.iloc[0, 3])
profit_rfc = get_business_profit(y_test, models_pred_proba_values[1], pivot_metric_table.iloc[1, 3])
profit_gbc = get_business_profit(y_test, models_pred_proba_values[2], pivot_metric_table.iloc[2, 3])

In [25]:
profit_lr, profit_rfc, profit_gbc

(-106, 146, 62)

- Также подсчитаем экономическую эффективность моделей с порогом срабатывания, соответствующим наибольшей прибыли.

In [26]:
def get_best_business_profit_th(y_test, y_preds, n_thresholds=20,
                                retain_cost=1, spec_profit=2):
    best_profit = -np.inf
    opt_th = None
    best_cm = None
    
    for th in np.linspace(0, 1, n_thresholds + 1):
        
        cnf_matrix = confusion_matrix(y_test, y_preds > th)
        TN = cnf_matrix[0][0]
        FN = cnf_matrix[1][0]
        TP = cnf_matrix[1][1]
        FP = cnf_matrix[0][1]
    
        profit = spec_profit * TP - retain_cost * (TP + FP)
        
        if profit > best_profit:
            opt_th = th
            best_profit = profit
            best_cm = cnf_matrix
    
    return opt_th, best_profit, (best_cm[1][1], best_cm[0][1], best_cm[1][0], best_cm[0][0])

In [27]:
opt_th_lr = get_best_business_profit_th(y_test, models_pred_proba_values[0], 100)
opt_th_rfc = get_best_business_profit_th(y_test, models_pred_proba_values[1], 100)
opt_th_gbc = get_best_business_profit_th(y_test, models_pred_proba_values[2], 100)

In [28]:
print(f'Для модели логистической регрессии максимальная прибыль - {opt_th_lr[1]}, при пороге - {opt_th_lr[0]:.2f}')
print(f'Для модели случайного леса максимальная прибыль - {opt_th_rfc[1]}, при пороге - {opt_th_rfc[0]:.2f}')
print(f'Для модели градиентного бустинга максимальная прибыль - {opt_th_gbc[1]}, при пороге - {opt_th_gbc[0]:.2f}')

Для модели логистической регрессии максимальная прибыль - 50, при пороге - 0.49
Для модели случайного леса максимальная прибыль - 170, при пороге - 0.68
Для модели градиентного бустинга максимальная прибыль - 172, при пороге - 0.54
